<a href="https://colab.research.google.com/github/petermr/semanticClimate/blob/main/abbreviation/Codes/climate_abbreviation_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/petermr/pyami.git

Cloning into 'pyami'...
remote: Enumerating objects: 6198, done.
remote: Counting objects: 100% (690/690), done.
remote: Compressing objects: 100% (369/369), done.
remote: Total 6198 (delta 370), reused 623 (delta 305), pack-reused 5508
Receiving objects: 100% (6198/6198), 172.48 MiB | 30.51 MiB/s, done.
Resolving deltas: 100% (2674/2674), done.


In [ ]:
!pip install abbreviations
!pip install scispacy==0.3.0
!python -m spacy download en_core_web_sm
#!pip install spacy==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 71 kB 9.4 MB/s 
     |████████████████████████████████| 13.5 MB 46.0 MB/s 
     |████████████████████████████████| 10.4 MB 54.2 MB/s 
     |████████████████████████████████| 188 kB 55.8 MB/s 
     |████████████████████████████████| 184 kB 55.0 MB/s 
     |████████████████████████████████| 1.0 MB 54.1 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.3
    Uninstalling srsly-2.4.3:
      Successfully uninstalled srsly-2.4.3
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.7
    Uninstalling catalogue-2.0.7:
      Successfully uninstalled catalogue-2.0.7
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninst

In [ ]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [ ]:
!python -m spacy download en_core_web_sm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 24.7 MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.3.1-py3-none-any.whl size=12047105 sha256=08478412ecef97ba028e784d8775efa7951c0a07f6f511f4ec01b30d55489d39
  Stored in directory: /root/.cache/pip/wheels/b7/0d/f0/7ecae8427c515065d75410989e15e5785dd3975fe06e795cd9
Successfully built en-core-web-sm
  Attempting uninstall: en-core-web-sm
    Found existing installation: en-core-web-sm 3.3.0
    Uninstalling en-core-web-sm-3.3.0:
      Successfully uninstalled en-core-web-sm-3.3.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
# spacy.load('en_core_web_sm')

In [ ]:
import spacy
import scispacy
from bs4 import BeautifulSoup
from abbreviations import schwartz_hearst
from scispacy.abbreviation import AbbreviationDetector

import os
dict_abbreviation = {}
for i in range(1,300):
  path = './'
  head_folder = path + f'Pages_climate_report/page_{i}' +'/'
  if not os.path.exists(head_folder):
    os.makedirs(head_folder)
  print(f'{i}---------------')
  with open(f'/content/pyami/temp/html/page.{i}.html', 'r') as f:
      html = f.read()
      soup = BeautifulSoup(html, features="html.parser")

      # kill all script and style elements
      for script in soup(["script", "style"]):
          script.extract()    # rip it out

      # get text
      text = soup.get_text()

      # break into lines and remove leading and trailing space on each
      lines = (line.strip() for line in text.splitlines())
      # break multi-headlines into a line each
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      # drop blank lines
      #text_write = '\n'.join(chunk for chunk in chunks if chunk)
      text = '\n'.join(chunk for chunk in chunks if chunk)

      #print(text)
      pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text= text)
      #print('pairs',pairs)
      
      TEXT_ = f'page_{i}_text.txt'
      DICTIONARY_SH = f'page_{i}_dictionary_SH.txt'
      DICTIONARY_Spacy= f'page_{i}_dictionary_Spacy.txt'
      with open(head_folder + TEXT_, 'w') as file:
          file.write(text)
      
      with open(head_folder + DICTIONARY_SH, 'w') as file:
          for key, value in pairs.items():
            #print(key, value )
            file.write('{'+str(key)+' : '+ str(value)+"}")
            file.write('\n')
      

      nlp = spacy.load('en_core_web_sm')
      abbreviation_pipe = AbbreviationDetector(nlp)
      nlp.add_pipe(abbreviation_pipe)
      doc = nlp(text)
      #print(doc._._)
      print(doc._.abbreviations)
      for abrv in doc._.abbreviations:
        new_dict  ={}
        print(abrv) 
        abrv_ = str(abrv)
        #print('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")
        if abrv_ not in dict_abbreviation:
          print('key_not_present')
          new_dict['long_form']= abrv._.long_form
          new_dict['count'] = int(1)
          dict_abbreviation[str(abrv)] = new_dict
          
        else:  
          print('key_present')
          print(dict_abbreviation[str(abrv)]['count'])
          #c = dict_abbreviation[str(abrv)]['count']
          dict_abbreviation[str(abrv)]['count'] = int(dict_abbreviation[str(abrv)]['count'])+ 1
          continue
        print(dict_abbreviation)  
      # altered_tok = [tok.text for tok in doc]
      with open(head_folder + DICTIONARY_Spacy, 'w') as file:
          for abrv in doc._.abbreviations:
            
            #print(abrv ,abrv._.long_form)
            #print(key, value )
            file.write('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")  
            file.write('\n')    

In [81]:
import spacy
import scispacy
from bs4 import BeautifulSoup
from abbreviations import schwartz_hearst
from scispacy.abbreviation import AbbreviationDetector
import re
import os
dict_abbreviation = {}
for i in range(1,20):
  path = './'
  head_folder = path + f'Pages_climate_report/page_{i}' +'/'
  if not os.path.exists(head_folder):
    os.makedirs(head_folder)
  #print(f'{i}---------------')
  with open(f'/content/pyami/temp/html/page.{i}.html', 'r') as f:
      html = f.read()
      soup = BeautifulSoup(html, features="html.parser")

      # kill all script and style elements
      for script in soup(["script", "style"]):
          script.extract()    # rip it out

      # get text
      text = soup.get_text()

      # break into lines and remove leading and trailing space on each
      lines = (line.strip() for line in text.splitlines())
      # break multi-headlines into a line each
      chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
      # drop blank lines
      #text_write = '\n'.join(chunk for chunk in chunks if chunk)
      text = ' '.join(chunk for chunk in chunks if chunk)

      #print(text)
      pairs = schwartz_hearst.extract_abbreviation_definition_pairs(doc_text= text)
      #print('pairs',pairs)
      
      TEXT_ = f'page_{i}_text.txt'
      DICTIONARY_SH = f'page_{i}_dictionary_SH.txt'
      DICTIONARY_Spacy= f'page_{i}_dictionary_Spacy.txt'
      with open(head_folder + TEXT_, 'w') as file:
          file.write(text)
      
      with open(head_folder + DICTIONARY_SH, 'w') as file:
          for key, value in pairs.items():
            #print(key, value )
            file.write('{'+str(key)+' : '+ str(value)+"}")
            file.write('\n')
      

      nlp = spacy.load('en_core_web_sm')
      abbreviation_pipe = AbbreviationDetector(nlp)
      nlp.add_pipe(abbreviation_pipe)
      doc = nlp(text)
      #print(doc._._)
      # print(doc._.abbreviations)
      for abrv in doc._.abbreviations:
        new_dict  ={}
        # print(abrv) 
        abrv_ = str(abrv)
        fullform = str(abrv._.long_form)
        if '(' in fullform or ')' in fullform :
          continue
        else:  
            #print('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")
            if abrv_ not in dict_abbreviation:
              # print('key_not_present')
              new_dict['long_form']= abrv._.long_form
              new_dict['count'] = int(1)
              dict_abbreviation[str(abrv)] = new_dict
              
            else:  
              # print('key_present')
              # print(dict_abbreviation[str(abrv)]['count'])
              #c = dict_abbreviation[str(abrv)]['count']
              dict_abbreviation[str(abrv)]['count'] = int(dict_abbreviation[str(abrv)]['count'])+ 1
              continue
            # print(dict_abbreviation)  
      # altered_tok = [tok.text for tok in doc]
      with open(head_folder + DICTIONARY_Spacy, 'w') as file:
          for abrv in doc._.abbreviations:
            
            #print(abrv ,abrv._.long_form)
            #print(key, value )
            file.write('{'+str(abrv)+' : '+ str(abrv._.long_form)+"}")  
            file.write('\n')    

In [96]:
from IPython.display import HTML
import pandas as pd
import requests
def wikidata_lookup(row):
    query = row['long_form']
    params	= {
            "action"		: "wbsearchentities",
            "search"		: query,
            "language"	: "en",
            "format"		: "json"
          }
    data	= requests.get("https://www.wikidata.org/w/api.php",params=params)
    result = data.json()
    hit_list = []
    for hit in result['search']:
      try:
        if "scientific article" not in hit["description"]:
            hit_list.append(hit["url"])
      except:
            hit_list.append(hit["url"])
    return hit_list



entry = {'abbreviation':[''],'long_form':[''],'count':[0]}
df = pd.DataFrame(entry)
for l in dict_abbreviation:
  entry = {'abbreviation':str(l),'long_form':str(dict_abbreviation[l]['long_form']),'count':dict_abbreviation[l]['count']}
  df = df.append(entry, ignore_index= True)
df = df.drop(0)
df = df.reset_index(drop=True)
df_copy = df.copy()
#print(df) 

df['wiki_search_links'] = df.apply(lambda row: wikidata_lookup(row), axis =1)
def make_clickable(val):
    #print(type(val))
    l = [f'<a target="_blank" href="{val_}">{val_}</a>' for val_ in val]
    return l

df.style.format({'wiki_search_links': make_clickable})

,abbreviation,long_form,count,wiki_search_links
0,France,Franck Lecocq,3,['//www.wikidata.org/wiki/Q58366013']
1,WG III,Working Group III,2,[]
2,TS,Technical Summary,1,['//www.wikidata.org/wiki/Q59447700']
3,UNFCCC,UN Framework Convention on Climate Change,3,[]
4,SDGs,Sustainable Development Goals,1,"['//www.wikidata.org/wiki/Q7649586', '//www.wikidata.org/wiki/Q22547988', '//www.wikidata.org/wiki/Q61794491']"
5,GHG,greenhouse gas,1,"['//www.wikidata.org/wiki/Q167336', '//www.wikidata.org/wiki/Q3588927', '//www.wikidata.org/wiki/Q925312']"
6,SPM,Summary for Policymakers,3,"['//www.wikidata.org/wiki/Q57831162', '//www.wikidata.org/wiki/Q60363253', '//www.wikidata.org/wiki/Q57023141', '//www.wikidata.org/wiki/Q108146077', '//www.wikidata.org/wiki/Q57001687']"
7,HFCs,hydrofluorocarbons,2,"['//www.wikidata.org/wiki/Q59100158', '//www.wikidata.org/wiki/Q1031943']"
8,PFCs,perfluorocarbons,2,['//www.wikidata.org/wiki/Q56989783']
9,GWP100,Global Warming Potential with a time horizon of 100 years,1,[]


In [97]:
def make_hyperlink(val):
    #val = row['wiki_search_links'] 
    url = "{}"
    l = ['=HYPERLINK("%s", "%s")' % (url.format(val_), val_) for val_ in val]
    return l
df['wiki_lookup_links'] = df['wiki_search_links'].apply(lambda x: make_hyperlink(x))
df_ = df.drop(columns = ['wiki_search_links'],axis = 1)
print(df_.wiki_lookup_links)
df.to_excel('gg.xlsx')

0     [=HYPERLINK("//www.wikidata.org/wiki/Q58366013...
1                                                    []
2     [=HYPERLINK("//www.wikidata.org/wiki/Q59447700...
3                                                    []
4     [=HYPERLINK("//www.wikidata.org/wiki/Q7649586"...
5     [=HYPERLINK("//www.wikidata.org/wiki/Q167336",...
6     [=HYPERLINK("//www.wikidata.org/wiki/Q57831162...
7     [=HYPERLINK("//www.wikidata.org/wiki/Q59100158...
8     [=HYPERLINK("//www.wikidata.org/wiki/Q56989783...
9                                                    []
10    [=HYPERLINK("//www.wikidata.org/wiki/Q58231661...
11    [=HYPERLINK("//www.wikidata.org/wiki/Q752401",...
12    [=HYPERLINK("//www.wikidata.org/wiki/Q1434887"...
13    [=HYPERLINK("//www.wikidata.org/wiki/Q57076275...
14                                                   []
15    [=HYPERLINK("//www.wikidata.org/wiki/Q3817465"...
16    [=HYPERLINK("//www.wikidata.org/wiki/Q20051297...
Name: wiki_lookup_links, dtype: object


#using scispacy example

In [ ]:
!pip install scispacy==0.3.0
!pip install spacy==3.3.1
import spacy
import scispacy
print(scispacy.__version__)
from scispacy.abbreviation import AbbreviationDetector
#from scispacy.abbreviation import AbbreviationDetector

nlp = spacy.load("en_core_web_sm")

abbreviation_pipe = AbbreviationDetector(nlp)
nlp.add_pipe(abbreviation_pipe)

text = "StackOverflow (SO) is a question and answer site for professional and enthusiast programmers. SO rocks!"


doc = nlp(text)
altered_tok = [tok.text for tok in doc]
for abrv in doc._.abbreviations:
  print(abrv ,abrv._.long_form)
    
#replace_acronyms(text)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.2 MB 23.6 MB/s 
  Using cached thinc-8.0.17-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (660 kB)
  Using cached srsly-2.4.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (457 kB)
  Using cached catalogue-2.0.7-py3-none-any.whl (17 kB)
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.5
    Uninstalling thinc-7.4.5:
      Successfully uninstalled thinc-7.4.5
  Attempting uninstall: spacy
    Found

0.3.0


/usr/local/lib/python3.7/dist-packages/spacy/util.py:730: UserWarning: [W095] Model 'en_core_web_sm' (3.3.0) was trained with spaCy v3.3 and may not be 100% compatible with the current version (3.1.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  return device


RegistryError: ignored